# Illustration of how multiple synaptic inputs tend to form dipolar LFPs

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, cmap_v_e
from brainsignals.neural_simulations import return_hay_cell
import neuron

np.random.seed(12345)

# Create a grid of measurement locations, in (um)
grid_x, grid_z = np.mgrid[-650:651:20, -1000:1600:20]
grid_y = np.ones(grid_x.shape) * 0

sigma = 0.3  # S/m

# Define electrode parameters
grid_elec_params = {
    'sigma': sigma,      # extracellular conductivity
    'x': grid_x.flatten(),  # electrode positions
    'y': grid_y.flatten(),
    'z': grid_z.flatten(),
    'method': 'linesource'
}

def insert_synaptic_input(idx, cell, syn_scale):

    synapse_parameters = {'e': 0., # reversal potential
                          'weight': 0.002 * syn_scale, #  synapse weight
                          'record_current': True, # record synapse current
                          'syntype': 'Exp2Syn',
                          'tau1': 1, #Time constant, rise
                          'tau2': 3, #Time constant, decay
                          }
    synapse_parameters['idx'] = idx
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([1.]))
    return synapse, cell



### Running all simulations

In [ ]:
def get_rand_idx_length_norm(cell, section='allsec', nidx=1,
                           z_min=-1E6, z_max=1E6):
    """Return nidx segment indices in section with random probability
    normalized to the membrane area of segment on
    interval [z_min, z_max]

    Parameters
    ----------
    section: str
        String matching a section-name
    nidx: int
        Number of random indices
    z_min: float
        Depth filter
    z_max: float
        Depth filter

    Returns
    -------
    ndarray, dtype=int
        segment indices
    """
    from LFPy.alias_method import alias_method
    poss_idx = cell.get_idx(section=section, z_min=z_min, z_max=z_max)
    if nidx < 1:
        print('nidx < 1, returning empty array')
        return np.array([])
    elif poss_idx.size == 0:
        print('No possible segment idx match quiery - returning '
              'empty array')
        return np.array([])
    else:
        length = cell.length[poss_idx]
        length /= length.sum()
        return alias_method(poss_idx, length, nidx)

tstop = 10
dt = 2**-6
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)

chosen_basal_idxs = np.array([129, 500, 95])#, 129,])
basal_idxs = cell.get_rand_idx_area_norm(section='allsec', z_max=100,
                                   z_min=-1e9, nidx=1000)
apic_idxs = cell.get_rand_idx_area_norm(section='allsec', z_max=1e9,
                                   z_min=700, nidx=1000)
uniform_idxs_w = cell.get_rand_idx_area_norm(section='allsec', z_max=1e9,
                                   z_min=-1e9, nidx=1000)
uniform_idxs_l = get_rand_idx_length_norm(cell, section='allsec', z_max=1e9,
                                   z_min=-1e9, nidx=1000)
syn_idxs_list = [[129], [500], [95], 
                basal_idxs,
                apic_idxs,
                uniform_idxs_w,
                uniform_idxs_l,
                uniform_idxs_w,
                ]

grid_electrode = LFPy.RecExtElectrode(cell, **grid_elec_params)
LFPs = []

for sim_idx, syn_idxs in enumerate(syn_idxs_list):
    cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=True)
    if sim_idx == len(syn_idxs_list) -1:
        comp_idx = 0
        for sec in cell.allseclist:
            for seg in sec:
                if cell.z[comp_idx].mean() > 500:
                    seg.g_pas *= 60
                comp_idx += 1       

    for syn_idx in syn_idxs:
        syn_scale = 1 / len(syn_idxs)
        syn, cell = insert_synaptic_input(syn_idx, cell, syn_scale)
    cell.simulate(rec_imem=True, rec_vmem=True)
    print("Max dV: ", np.max(np.abs(cell.vmem - cell.vmem[:, 0, None])))
    LFP = 1000 * grid_electrode.get_transformation_matrix() @ cell.imem
    LFPs.append(LFP)
    cell.__del__()



### Plotting results

In [ ]:
def plot_grid_LFP(cell, LFP, ax, syn_idxs, scale_max=None):
    
    max_amp_elec_idx = np.argmax(np.max(np.abs(LFP), axis=1))
    max_amp_t_idx = np.argmax(np.abs(LFP[max_amp_elec_idx, :]))

    max_amp_LFP = np.max(np.abs(LFP))
    print(max_amp_LFP)
    if not max_amp_LFP == np.abs(LFP[max_amp_elec_idx, max_amp_t_idx]):
        raise RuntimeError("Wrong with chosen max value")

    LFP = LFP[:, max_amp_t_idx].reshape(grid_x.shape)
    num = 11
    levels = np.logspace(-2.3, 0, num=num)

    scale_max = np.max(np.abs(LFP)) if scale_max is None else scale_max
    
    levels_norm = scale_max * np.concatenate((-levels[::-1], levels))

    colors_from_map = [cmap_v_e(i/(len(levels_norm) - 2))
                       for i in range(len(levels_norm) - 1)]
    colors_from_map[num - 1] = (1.0, 1.0, 1.0, 1.0)

    ep_intervals = ax.contourf(grid_x, grid_z, LFP,
                                   zorder=2, colors=colors_from_map,
                                   levels=levels_norm, extend='both')

    ax.contour(grid_x, grid_z, LFP, colors='k', linewidths=(1), zorder=2,
                   levels=levels_norm)
    ax.plot(cell.x.T, cell.z.T, lw=1, c='gray')
    if len(syn_idxs) == 1:
        ms = 5
    else:
        ms = 1.5
    [ax.plot(cell.x[syn_idx].mean(), cell.z[syn_idx].mean(), marker='o', c='b',
                 ms=ms, mec='none', mew=0.2)
     for syn_idx in syn_idxs]
    
    return ep_intervals

scale_max = 1

ax_lfp_dict = dict(aspect=1, frameon=False, xticks=[], yticks=[],
                   ylim=[np.min(grid_z) - 2, np.max(grid_z) + 2],
                   xlim=[np.min(grid_x) - 2, np.max(grid_x) + 2])
    
plt.close("all")
fig = plt.figure(figsize=[6, 3.8])
fig.subplots_adjust(bottom=-0.05, top=1.08, right=1.03,
                    left=-0.03, wspace=-0.02, hspace=-0.1)
num_cols = 5
num_rows = 2
ax_jump = 0

imgs = []
axes_to_mark = []
for i, syn_idxs in enumerate(syn_idxs_list):
    syn_scale = scale_max#single_syn_scalemax if len(syn_idxs) == 1 else multi_syn_scalemax
    if len(syn_idxs) > 1:
        ax_jump = 2
    ax = fig.add_subplot(num_rows, num_cols, i + 1 + ax_jump, **ax_lfp_dict)
    axes_to_mark.append(ax)    
    img = plot_grid_LFP(cell, LFPs[i], ax, syn_idxs, scale_max=syn_scale)
    imgs.append(img)
    
mark_subplots(axes_to_mark, xpos=0.15, ypos=0.82)

cax = fig.add_axes([0.6, 0.55, 0.01, 0.42], frameon=False)
cbar = fig.colorbar(imgs[-1], cax=cax)
cbar.set_label('V$_{\mathrm{e}}$ (µV)', labelpad=4)
cbar.set_ticks(np.array([-1, -0.1, -0.01, 0.01, 0.1, 1]) * scale_max)

fig.savefig("fig_chosen_dipoles.pdf")